In [1]:
from textblob import Word
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_context("notebook")

In [ ]:
!pip install contractions

In [ ]:
!pip install aksharamukha

In [ ]:
import contractions as contracs
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
import math
import re,string,unicodedata
from sklearn.model_selection import train_test_split
from textblob import TextBlob

In [ ]:
import json
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/"

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d preetviradiya/english-hindi-dataset -p /content
!unzip -q /content/english-hindi-dataset.zip -d dataset

Dataset URL: https://www.kaggle.com/datasets/preetviradiya/english-hindi-dataset
License(s): GNU Free Documentation License 1.3
  0% 0.00/13.9M [00:00<?, ?B/s]
100% 13.9M/13.9M [00:00<00:00, 476MB/s]
replace dataset/Dataset_English_Hindi.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
data=pd.read_csv("dataset/Dataset_English_Hindi.csv")

In [ ]:
data.sample(5)

,English,Hindi
45275,This duty-respecter enhance people to do their...,2. यह कार्यपालिका विधायिका को उनके संवैधानिक क...
38392,Valabhacharya did promotion-spread of Pushtimarg.,वल्लभाचार्य ने पुष्टिमार्ग का प्रचार-प्रसार किया।
101516,Sat Bull. In this the male straights his legs ...,बैठा सांड़ इसमें पुरुष बिस्तर में अपने पांव सी...
9805,"At 10.27 a.m . , I was brought to the mehmankh...",उस दिन सुबह 10.27 बजे मुज्हो मेहमानखाने में लय...
30261,"how it looks - the world, in 1960, it starts t...","तो, 1960 में विश्व किस तरह दिखता है, यह जानना ..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130476 entries, 0 to 130475
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   English  130474 non-null  object
 1   Hindi    130164 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [ ]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

In [ ]:
data['English']=data['English'].str.lower()

In [ ]:
import re
def remhtml(text):   #html tags
    pat=r'[^a-zA-Z0-9\s]'
    text=re.sub(pat,'',text)
    return text
data['English']=data['English'].apply(remhtml)

In [ ]:
def remurl(text):            #urls
    pat=re.compile(r'https?://\S+|www\.\S+')
    return pat.sub(r'',text)
data['English']=data['English'].apply(remurl)
data['Hindi']=data['Hindi'].apply(remurl)

In [ ]:
def expandcontracs(text):        #no shortcuts
    expanded=contracs.fix(text)
    return expanded
data['English']=data['English'].apply(expandcontracs)

In [ ]:
def prep2(text,lang='english'):
    if not isinstance(text,str):
        return text
    if lang=='english':
        pat=re.compile(r'[^a-zA-Z0-9\s]')    #rem all except spaces,letters,digits
        return pat.sub(r'',text)
    elif lang=='hindi':
        pat=re.compile(r'[^\u0900-\u097F\s]')    #rem anything not in hindi devnagari and spaces
        return pat.sub(r'',text)

data['English']=data['English'].apply(lambda x:prep2(x,lang='english'))
data['Hindi']=data['Hindi'].apply(lambda x:prep2(x,lang='hindi'))

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from textblob import Word

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
hinstopwords = set([
    "और", "के", "का", "की", "से", "है", "हैं", "यह", "था", "थे",
    "हो", "में", "पर", "को", "उनका", "उनकी", "उनके", "उसका", "उसकी",
    "इसके", "इसके", "इसका", "इसकी", "जो", "कि", "था", "थे", "तो",
    "ही", "भी", "लेकिन", "क्योंकि", "जैसे", "जब", "तक", "अगर", "या"
])

In [ ]:
def prep(text, lang="english"):
    if not isinstance(text, str):   #return empty string if input is not a str
        return ""

    if lang=="english":
        stop=set(stopwords.words("english"))
        words=text.split()
        words=[Word(w).lemmatize() for w in words if w not in stop]     #lematize eachword and then remove stopwords
    elif lang=="hindi":
        stop=hinstopwords
        words=text.split()
        words=[w for w in words if w not in stop]    #keep only words that are not stopwords
    else:
        words=text.split()
    return " ".join(words)    #join back

data['English']=data['English'].apply(lambda x: prep(x,lang='english'))
data['Hindi']=data['Hindi'].apply(lambda x: prep(x,lang='hindi'))


In [ ]:
data.sample(5)

,English,Hindi
31349,wallace remarked necessary issue prospectus ju...,जैसा वालस ने लिखा एक जूट मिल केवल अपना प्रास्प...
70724,bjorn rorslett norway sent,मुझे नार्वे ब्जोर्न रोर्सलेट ने भेजी
14011,people education,सिर्फ लोग शिक्षित
81725,sadness,ये दुख सिर्फ़ एक
85309,fence manhattan bridge,बाहरी दीवार चढ़ गया


In [ ]:
data=data[data['English'].str.len()<=50]   #only strs <50 len

In [ ]:
data.shape

(73243, 2)

In [ ]:
data.to_csv("datasethindi.csv",index=False)

In [ ]:
import re
import pandas as pd
from aksharamukha import transliterate
def hindi2hinglish(text):
    if pd.isna(text):
        return ""
    out=transliterate.process('Devanagari','ITRANS',text,nativize=True)   #aksharamukha transliterator, to covert devanagari into hinglish like
    out=normalize(out)   #normalization to clean
    return out.lower()
def normalize(text):
    rules={           #to fix common inconsistencies
        'aa': 'a',
        'ii': 'i',
        'uu': 'u',
        'ee': 'i',
        'oo': 'u',
        'shh': 'sh',
        'sh': 'sh',
        'chh': 'ch',
        'khh': 'kh',
        'ghh': 'gh',
        'ph': 'f',
        'dhh': 'dh',
        'ri': 'ri',
        'ya': 'ya',
        'aa ': 'a ',
    }
    for k,v in rules.items():
        text=text.replace(k,v)
    text=text.lower()
    text=re.sub(r"\.\.+",".",text)        #multiple dots into one
    text=re.sub(r"[^a-z0-9\s\.,!?]","",text) #only a-z,digits,spaces and puncs
    text=re.sub(r"\s+"," ", text).strip()    #multiple spaces into one
    text=re.sub(r'[āīūṛṅñṭḍṇśṣ]','',text)   #rem special diacritics from hinglish text
    return text

In [ ]:
def process2(text):
    text=hindi2hinglish(text)
    text=normalize(text)
    return text

data["Hinglish"]=data["Hindi"].apply(process2)

In [ ]:
data=data.drop(columns=["Hindi"])

In [ ]:
data.shape

(73243, 2)

In [ ]:
data.sample(5)

,English,Hinglish
89080,could use picture bee followed picture leaf,apa madhumakkhi bi tasvira piche eka patti lif...
30543,13 year 10 month age,usa samaya balaka umra thi teraha sala dasa ma...
65931,insect see green blue ultraviolet,ki.de hara nila parabaimgani dekhate
1442,must send doctor,tumhem da.cktara bulana hoga.
60850,fiction describe alien creature,parikalpana vo paragrahi jiva tina pairom vale


In [ ]:
data.to_csv("datasethinglish.csv", index=False)